# Scraping NextDoor NL

The script is used to scrape Nextdoor feed.

In [1]:
import bs4
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Scraping on NextDoor

It is not possible to use scrapy and bs4 alone. Selenium will be used to simulate scrolling.

In [2]:
import selenium
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#initiate chrome driver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get("https://nextdoor.nl/news_feed/")

#Manual login and go to neighbourhood page

[WDM] - Downloading:  94%|█████████▍| 5.93M/6.29M [00:00<00:00, 10.9MB/s]

[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:20<00:00, 10.9MB/s]

In [4]:
import time 

while True:
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)


KeyboardInterrupt: 

In [105]:
#change Dutch spelling into english
def dutch_to_eng(date):
    eng_date = date.replace('mrt','mar')
    eng_date = date.replace('mei','may')
    eng_date = date.replace('okt','oct')

    return eng_date

In [90]:
#parse date and time of post 
def changedate(date):
    changed_date = datetime.datetime.strptime(date, '%d %b.').strftime('%d/%m/2022')

    return changed_date

In [106]:
date = 'mrt'

dutch_to_eng(date)


'mrt'

In [98]:
#get links from result lists
from bs4 import element
from pandas import DataFrame
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

post_grid = driver.find_elements(By.CLASS_NAME, "css-15wtqd7") #pass list of individual posts to post_grid

post = []
text = []
comment_count = []
post_date = []


for i in range(len(post_grid)):
    try:
        while i == 0: #click on all "more comments" button for the first iteration
            post_grid[i].find_element(By.CLASS_NAME, "css-adjc8x").click() #class css-adjc8x is the class name for more comments

        post_id = post_grid[i].get_attribute("id") #get the post id 

        #get post text
        post_text = post_grid[i].find_element(By.CLASS_NAME, "content-body").text #get body text from post[i]

        #get post comment counts 
        comment = post_grid[i].find_element(By.CLASS_NAME, "comment-container").find_elements(By.CLASS_NAME, "Linkify") #return list of comment (if any)

        temp_comment_count = len(comment)
            
        #get post date
        byline = post_grid[i].find_elements(By.CLASS_NAME, "post-byline-redesign") #getting two elements from byline

        date = byline[1].text #pass the second element to date 

        dutch_to_eng(date) #change dutch date to eng date

        changedate(eng_date) #change date to eng date

        post.append(post_id) 
        text.append(post_text)
        comment_count.append(temp_comment_count)
        post_date.append(date)

        print(f"Processed post: {i}")
        

    except NoSuchElementException:
        pass

NameError: name 'eng_date' is not defined

In [63]:
#Create data frame
all_post = pd.DataFrame(columns=['ID','Text','Comment Count'])

all_post['ID'] = post
all_post['Text'] = text
all_post['Comment Count'] = comment_count
all_post['Date'] = post_date

all_post 

,ID,Text,Comment Count,Date
0,s_17592212454729,Douvle Snak NC is zojuist lid geworden van Da...,0,2 dagen geleden
1,s_17592212390557,Hallo allemaal. 👋 Ik ben Marieke. Ik woon in d...,9,5 dagen geleden
2,s_17592212424616,"Ik ben op zoek naar behang, liefst een lichte ...",1,3 dagen geleden
3,s_17592212409493,"Ik ben Sofi,\nWij bieden u onze schoonmaakser...",0,4 dagen geleden
4,s_17592212390716,Onze kat Jopie is vermist sinds 4 oktober. We ...,1,5 dagen geleden
...,...,...,...,...
84,s_17592208794661,"Hallo lieve vrienden en buren, ik ben een nieu...",0,20 apr.
85,s_17592208645632,Klimaatwandeling Amsterdam-Oost. De Raad van K...,1,15 apr.
86,s_17592208538359,"Poppenkast OPGEPAST. Traditionele, maar hedend...",1,11 apr.
87,s_17592208500642,Hi neighbors. I’m Simona. 👋. Hallo allemaal! I...,1,10 apr.


In [64]:
#Specific function for actual comment as it requires a set of different data structure 

comment = []
comment_post_id = []

for i in range(len(post_grid)):
    try:
        post_id = post_grid[i].get_attribute("id") #get post id

        temp_comment_count = post_grid[i].find_element(By.CLASS_NAME, "comment-container").find_elements(By.CLASS_NAME, "Linkify") #return list of comment (if any)
        
        if temp_comment_count != 0:
            for comment_num in range(len(temp_comment_count)):
                comment_text = post_grid[i].find_element(By.CLASS_NAME, "comment-container").find_elements(By.CLASS_NAME, "Linkify")[comment_num].text

                comment.append(comment_text)
                comment_post_id.append(post_id)
    except NoSuchElementException:
        pass

In [67]:
#Create comment texts dataframe
detailed_comment = pd.DataFrame(columns=["ID","Comment"])

detailed_comment['ID'] = comment_post_id
detailed_comment['Comment'] = comment

detailed_comment

,ID,Comment
0,s_17592212390557,"Hi Marieke, is dat de kat waar een foto van op..."
1,s_17592212390557,"Ja, dat klopt. Dat is Jopie. Heb je hem gezien?"
2,s_17592212390557,Ik loop iedere ochtend rond 7 uur wat rondes ...
3,s_17592212390557,Dank voor de tip. We gaan daar even kijken!
4,s_17592212390557,"Hallo Desiree, Zijn er specifieke plekken waar..."
5,s_17592212390557,Als je van de lineus kant hè park ingaat links...
6,s_17592212390557,"Ok, dank je. Daar gaan we morgenochtend even k..."
7,s_17592212390557,Hij was zijn bandje net kwijt.. geen bandje dus
8,s_17592212390557,Ok ik houd het in de gaten voor je. Sterkte!
9,s_17592212424616,Tip: Ik zag wat rolletjes behang liggen in de ...


In [68]:
#Merge dataframe

all_post.merge(detailed_comment, on = "ID", how = "left")

,ID,Text,Comment Count,Date,Comment
0,s_17592212454729,Douvle Snak NC is zojuist lid geworden van Da...,0,2 dagen geleden,NaN
1,s_17592212390557,Hallo allemaal. 👋 Ik ben Marieke. Ik woon in d...,9,5 dagen geleden,"Hi Marieke, is dat de kat waar een foto van op..."
2,s_17592212390557,Hallo allemaal. 👋 Ik ben Marieke. Ik woon in d...,9,5 dagen geleden,"Ja, dat klopt. Dat is Jopie. Heb je hem gezien?"
3,s_17592212390557,Hallo allemaal. 👋 Ik ben Marieke. Ik woon in d...,9,5 dagen geleden,Ik loop iedere ochtend rond 7 uur wat rondes ...
4,s_17592212390557,Hallo allemaal. 👋 Ik ben Marieke. Ik woon in d...,9,5 dagen geleden,Dank voor de tip. We gaan daar even kijken!
...,...,...,...,...,...
92,s_17592208794661,"Hallo lieve vrienden en buren, ik ben een nieu...",0,20 apr.,NaN
93,s_17592208645632,Klimaatwandeling Amsterdam-Oost. De Raad van K...,1,15 apr.,"Lijkt me een mooi initiatief, maar helaas voor..."
94,s_17592208538359,"Poppenkast OPGEPAST. Traditionele, maar hedend...",1,11 apr.,"Hi Annemieke, dank je wel voor je reactie. Het..."
95,s_17592208500642,Hi neighbors. I’m Simona. 👋. Hallo allemaal! I...,1,10 apr.,"Hey,..ik woon in de buurt sinds 2005!,..Tja,…A..."
